In [38]:
#Imports
from google.colab import drive
import pandas as pd
import keras
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense, LSTM, Flatten, Embedding, Dropout , Activation, GRU, Flatten, Input, Bidirectional, GlobalMaxPool1D, Convolution1D, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers

In [39]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
data = pd.read_csv('/content/drive/My Drive/Fixy/merged_ki_final_X.csv')
print(data.shape)
data.head()

(304244, 2)


,line,y
0,karşı tarafta x kapıya git.,1
1,- i̇ x kişilikmiş.,1
2,lisedeyken iç x kaçakçılığı yapardım.,1
3,"binbaşı, şüphem yok x geçmişindeki şiddet sonu...",0
4,"nayam-singh, arkady kolcheck'in sektörde x rak...",1


In [41]:
X = data['line']
y = data['y']

In [42]:
vocab_size = 15000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X)
list_tokenized_train = tokenizer.texts_to_sequences(X)

In [43]:
maxlen = 7
X_train_padded = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_train_padded, y, test_size=0.2, stratify=y)

In [45]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True, verbose=1)

In [65]:
embed_size = 128
model = Sequential()
model.add(Embedding(vocab_size, embed_size))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 128)         1920000   
_________________________________________________________________
bidirectional_15 (Bidirectio (None, None, 128)         98816     
_________________________________________________________________
bidirectional_16 (Bidirectio (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 2,060,097
Trainable params: 2,060,097
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])

batch_size = 10000
epochs = 6
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping_cb])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 219055 samples, validate on 24340 samples
Epoch 1/6
219055/219055 [==============================] - 6s 28us/step - loss: 0.6575 - auc: 0.6704 - accuracy: 0.6936 - val_loss: 0.5300 - val_auc: 0.7826 - val_accuracy: 0.8408
Epoch 2/6
219055/219055 [==============================] - 5s 21us/step - loss: 0.3211 - auc: 0.8414 - accuracy: 0.8873 - val_loss: 0.2366 - val_auc: 0.8800 - val_accuracy: 0.9086
Epoch 3/6
219055/219055 [==============================] - 5s 21us/step - loss: 0.1967 - auc: 0.9031 - accuracy: 0.9237 - val_loss: 0.2151 - val_auc: 0.9193 - val_accuracy: 0.9123
Epoch 4/6
219055/219055 [==============================] - 5s 21us/step - loss: 0.1713 - auc: 0.9302 - accuracy: 0.9341 - val_loss: 0.2117 - val_auc: 0.9384 - val_accuracy: 0.9119
Epoch 5/6
219055/219055 [==============================] - 5s 21us/step - loss: 0.1588 - auc: 0.9447 - accuracy: 0.9395 - val_loss: 0.2152 - val_auc: 0.9495 - val_accuracy: 0.9102
Epoch 6/6
219055/219055 [========================

In [71]:
pred = tokenizer.texts_to_sequences(["desem x böyle böyle oldu"])
print(pred)
maxlen = 7
padded_pred = pad_sequences(pred, maxlen=maxlen)
print(padded_pred)
model.predict(padded_pred)

#bu sene x yarışma çok zorlu geçecek
#duydum x unutmuşsun
#şöyle x sıçtık
#desem x böyle böyle oldu

[[3876, 1, 96, 96, 81]]
[[   0    0 3876    1   96   96   81]]


array([[0.00843348]], dtype=float32)

In [67]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

y_pred = model.predict_classes(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Data: %.2f%%" % (accuracy * 100.0))

print(confusion_matrix(y_test, y_pred ))
print(classification_report(y_test, y_pred, digits=4))
print(roc_auc_score(y_test, y_pred))

Accuracy on Test Data: 91.32%
[[27113  3311]
 [ 1968 28457]]
              precision    recall  f1-score   support

           0     0.9323    0.8912    0.9113     30424
           1     0.8958    0.9353    0.9151     30425

    accuracy                         0.9132     60849
   macro avg     0.9141    0.9132    0.9132     60849
weighted avg     0.9141    0.9132    0.9132     60849

0.913243897640815


In [72]:
model.save("Model_ki.h5")
print("Saved model to disk")

Saved model to disk


In [74]:
model.load_weights("/content/Model_ki.h5")

In [78]:
with open('tokenizer_ki.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)